# Identifying Significant COPD Genes Using Linear Regression 

## Purpose: 

* Load heterogeneous data sources
* Test associations selected based on Xiaowei's QTL analyses
* Test association of each gene with each COPD classification variable for each cell type
* Multiple-test correction
* Output table with all test results 

## Packages and Options

In [13]:
import pandas as pd
import numpy as np 
import glob

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import statsmodels.regression.linear_model
import statsmodels.stats.multitest

pd.options.display.max_rows=1000000
pd.options.display.max_columns=500

## Load Data

### Target Variables

In [2]:
target_matrices = []

# load matrices and fill blanks with nan
for matrix in sorted(glob.glob("/home/jve4pt/Manichaikul-PhD-Rotation/rna-seq-regression/output/target/*")): 
   
    tmp_df = pd.read_csv(
        matrix,
        sep="\t"
    ).set_index("Unnamed: 0").replace(["", " "], np.nan)
    
    # get cell type and set as name 
    tmp_df.name = matrix.split("/")[-1].split(".")[0].split("-")[0]
    tmp_df.name
    tmp_df.shape
    tmp_df.head()
    
    target_matrices.append(tmp_df)

'Monocyte_Exam_5'

(346, 6)

,pre_fev1,pre_fvc,pre_fev1fvc,post_fev1,post_fvc,post_fev1fvc
Unnamed: 0,,,,,,
NWD331470,NaN,NaN,NaN,NaN,NaN,NaN
NWD838347,1.759,2.355,74.692144373673,NaN,NaN,NaN
NWD283680,NaN,NaN,NaN,NaN,NaN,NaN
NWD496227,NaN,NaN,NaN,NaN,NaN,NaN
NWD420821,2.261,2.969,76.1535870663523,NaN,NaN,NaN


'PBMC_Exam_5'

(850, 6)

,pre_fev1,pre_fvc,pre_fev1fvc,post_fev1,post_fvc,post_fev1fvc
Unnamed: 0,,,,,,
NWD331470,NaN,NaN,NaN,NaN,NaN,NaN
NWD133576,NaN,NaN,NaN,NaN,NaN,NaN
NWD302101,NaN,NaN,NaN,NaN,NaN,NaN
NWD838347,1.759,2.355,74.692144373673,NaN,NaN,NaN
NWD264729,NaN,NaN,NaN,NaN,NaN,NaN


'T_cell_Exam_5'

(353, 6)

,pre_fev1,pre_fvc,pre_fev1fvc,post_fev1,post_fvc,post_fev1fvc
Unnamed: 0,,,,,,
NWD331470,NaN,NaN,NaN,NaN,NaN,NaN
NWD838347,1.759,2.355,74.692144373673,NaN,NaN,NaN
NWD283680,NaN,NaN,NaN,NaN,NaN,NaN
NWD420821,2.261,2.969,76.1535870663523,NaN,NaN,NaN
NWD638901,1.318,1.909,69.0413829229963,1.372,1.971,69.60933536276


### Covariates

In [3]:
covariate_matrices = []

# load matrices and fill blanks with nan
for matrix in sorted(glob.glob("/home/jve4pt/Manichaikul-PhD-Rotation/rna-seq-regression/output/covariates//*")): 
   
    tmp_df = pd.read_csv(
        matrix,
        sep="\t"
    ).set_index("Unnamed: 0").replace(["", " "], np.nan)
    
    # get cell type and set as name 
    tmp_df.name = matrix.split("/")[-1].split(".")[0].split("-")[0]
    tmp_df.name
    tmp_df.shape
    tmp_df.head()
    
    covariate_matrices.append(tmp_df)

'Monocyte_Exam_5'

(346, 56)

,age,sex,ht_cm,wt_kg,smoking_packyears,smoking_current,smoking_former,race_black,race_chinese,race_hispanic,race_native,race_others,race_white,age^2,ht_cm^2,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,InferredCov1,InferredCov2,InferredCov3,InferredCov4,InferredCov5,InferredCov6,InferredCov7,InferredCov8,InferredCov9,InferredCov10,InferredCov11,InferredCov12,InferredCov13,InferredCov14,InferredCov15,InferredCov16,InferredCov17,InferredCov18,InferredCov19,InferredCov20,InferredCov21,InferredCov22,InferredCov23,InferredCov24,InferredCov25,InferredCov26,InferredCov27,InferredCov28,InferredCov29,InferredCov30
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
NWD331470,77.0,1.0,158.3,91.40040,5.60,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,5929.0,25058.89,0.001333,0.006351,0.009273,0.000499,0.001695,-0.004290,0.005516,-0.002047,0.003131,-0.001841,0.000408,0.008957,0.015706,0.042628,-0.020999,-0.020670,-0.008807,-0.011506,-0.025096,-0.062135,-0.000179,0.083683,0.093949,0.005168,-0.057303,-0.006750,-0.105466,0.026402,-0.004576,-0.095270,-0.068875,-0.078650,-0.018101,0.042808,0.088160,0.032070,0.065956,0.063804,-0.010799,0.026043,0.087443
NWD838347,70.0,0.0,163.4,71.66880,24.78,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4900.0,26699.56,0.002261,-0.001662,-0.000558,0.000039,0.000018,0.002273,-0.001670,-0.000370,0.003888,0.000146,0.001026,-0.047761,0.058362,0.009556,-0.050034,-0.007238,-0.013260,0.059029,0.004162,-0.024531,0.018645,0.012871,-0.086288,-0.058938,-0.047777,-0.104149,-0.033986,0.058954,0.128375,-0.007764,-0.094265,0.033080,0.053943,-0.058609,-0.065562,0.047106,0.061745,-0.058382,0.001148,0.063844,-0.014274
NWD283680,77.0,0.0,156.8,70.30800,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5929.0,24586.24,0.002299,-0.001604,-0.000735,0.001416,0.002190,-0.001993,0.002143,0.000234,-0.008070,-0.002662,0.000313,-0.033265,-0.063524,0.008174,0.060593,0.049096,0.033341,0.055445,0.018947,0.043687,-0.025829,-0.015269,0.090913,0.051935,0.003507,0.054500,-0.055023,0.035402,-0.057874,-0.076134,-0.007955,0.058256,0.072242,0.047285,0.012751,0.016442,0.068699,0.010818,0.082827,0.007812,-0.016139
NWD496227,58.0,1.0,164.6,65.27304,61.58,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3364.0,27093.16,0.002369,-0.001689,-0.000867,0.001428,0.002522,-0.001239,0.000898,-0.001428,-0.005049,-0.001211,-0.000560,-0.028152,-0.034123,0.015064,0.060405,0.107458,-0.057786,0.033347,0.003128,0.010610,-0.035506,0.058446,-0.082613,-0.038286,0.093956,0.169138,0.049847,0.002884,0.075554,0.061193,0.010242,0.028568,-0.074561,0.039254,0.050816,0.067082,-0.000515,-0.138514,0.039030,-0.106134,0.031749
NWD420821,69.0,0.0,154.5,69.62760,0.00,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4761.0,23870.25,0.001329,0.004460,0.006931,-0.000199,0.000409,-0.002876,0.003032,-0.001861,0.003163,-0.003338,0.001002,-0.031820,-0.014172,0.058474,0.107694,-0.034027,-0.014747,0.000315,0.019139,0.083884,-0.001486,0.048308,-0.031755,-0.026437,0.066819,0.014255,-0.025266,-0.023918,-0.132586,-0.053560,0.061695,-0.013772,-0.081110,-0.018167,0.037213,-0.042580,0.173256,-0.074412,-0.022677,0.072744,-0.026444


'PBMC_Exam_5'

(850, 56)

,age,sex,ht_cm,wt_kg,smoking_packyears,smoking_current,smoking_former,race_black,race_chinese,race_hispanic,race_native,race_others,race_white,age^2,ht_cm^2,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,InferredCov1,InferredCov2,InferredCov3,InferredCov4,InferredCov5,InferredCov6,InferredCov7,InferredCov8,InferredCov9,InferredCov10,InferredCov11,InferredCov12,InferredCov13,InferredCov14,InferredCov15,InferredCov16,InferredCov17,InferredCov18,InferredCov19,InferredCov20,InferredCov21,InferredCov22,InferredCov23,InferredCov24,InferredCov25,InferredCov26,InferredCov27,InferredCov28,InferredCov29,InferredCov30
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
NWD331470,77.0,1.0,158.3,91.4004,5.60,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,5929.0,25058.89,0.001333,0.006351,0.009273,0.000499,0.001695,-0.004290,0.005516,-0.002047,0.003131,-0.001841,0.000408,-0.016531,0.021389,-0.007093,-0.009030,0.018956,-0.041270,0.047019,-0.010671,0.014834,0.041180,0.032549,-0.010475,0.030836,-0.007270,0.026989,-0.012802,0.025969,0.074676,-0.097949,-0.047847,-0.056101,0.067817,-0.008513,0.016477,0.019178,0.021043,-0.001567,0.012739,-0.021661,0.030757
NWD133576,69.0,1.0,187.6,90.2664,0.00,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4761.0,35193.76,0.002323,-0.001711,-0.000821,0.000911,0.001814,-0.000432,-0.000426,0.000145,0.001356,0.002580,0.000075,-0.018616,0.019134,0.002262,-0.022198,0.042012,0.005564,0.038360,-0.038189,-0.009368,0.017212,0.032560,-0.013100,0.004182,-0.039102,-0.083087,-0.031743,0.017137,-0.004303,0.001194,0.090485,-0.024339,0.029431,-0.009297,-0.015955,0.035380,0.049054,-0.021517,0.065846,-0.035764,-0.087970
NWD302101,87.0,0.0,160.5,54.6588,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7569.0,25760.25,0.002307,-0.001541,-0.000770,0.001028,0.001575,-0.001899,0.003010,-0.000059,-0.011193,-0.006435,0.001511,0.027519,-0.003869,-0.006178,0.019617,-0.015370,0.009938,-0.092423,0.122216,0.026003,-0.022831,-0.021640,-0.007357,0.051379,0.005893,-0.016062,0.000180,-0.097866,-0.057747,-0.080866,-0.006194,0.031609,-0.024620,0.011391,-0.078764,0.042386,-0.006648,-0.062983,0.041313,0.026158,-0.037687
NWD838347,70.0,0.0,163.4,71.6688,24.78,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4900.0,26699.56,0.002261,-0.001662,-0.000558,0.000039,0.000018,0.002273,-0.001670,-0.000370,0.003888,0.000146,0.001026,-0.020901,0.024301,0.022153,-0.037669,-0.011932,0.001810,0.030675,0.029627,-0.055695,-0.056049,-0.098353,0.073964,-0.018510,-0.088976,-0.075302,-0.105775,0.083273,-0.010062,0.175579,-0.129532,-0.183875,-0.055429,-0.097938,0.129734,-0.145657,0.005768,0.010597,0.102760,-0.024699,-0.138352
NWD264729,67.0,0.0,173.4,73.9368,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4489.0,30067.56,0.002330,-0.001671,-0.000870,0.001166,0.002109,-0.001416,0.001174,-0.000427,-0.002391,0.000257,-0.000790,-0.014680,-0.030278,-0.004421,-0.029180,0.031054,-0.075984,-0.059270,0.009909,0.046657,-0.037481,0.026605,-0.069632,-0.050175,-0.079759,-0.028212,-0.026700,0.019788,-0.007830,-0.018503,0.006483,-0.079485,0.010320,0.066171,-0.116573,-0.137902,0.016449,-0.029452,-0.039188,-0.195959,-0.086525


'T_cell_Exam_5'

(353, 56)

,age,sex,ht_cm,wt_kg,smoking_packyears,smoking_current,smoking_former,race_black,race_chinese,race_hispanic,race_native,race_others,race_white,age^2,ht_cm^2,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,InferredCov1,InferredCov2,InferredCov3,InferredCov4,InferredCov5,InferredCov6,InferredCov7,InferredCov8,InferredCov9,InferredCov10,InferredCov11,InferredCov12,InferredCov13,InferredCov14,InferredCov15,InferredCov16,InferredCov17,InferredCov18,InferredCov19,InferredCov20,InferredCov21,InferredCov22,InferredCov23,InferredCov24,InferredCov25,InferredCov26,InferredCov27,InferredCov28,InferredCov29,InferredCov30
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
NWD331470,77.0,1.0,158.3,91.4004,5.60,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,5929.0,25058.89,0.001333,0.006351,0.009273,0.000499,0.001695,-0.004290,0.005516,-0.002047,0.003131,-0.001841,0.000408,0.012913,0.015743,-0.017391,0.012124,-0.048268,-0.071135,-0.026182,0.015410,0.020431,-0.010921,-0.066185,0.057517,0.127341,0.032182,-0.055287,-0.015699,0.028204,-0.109985,-0.043114,-0.082227,0.128742,-0.024782,0.080862,-0.071816,-0.010468,0.031179,-0.096388,-0.083158,-0.009181,-0.021751
NWD838347,70.0,0.0,163.4,71.6688,24.78,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4900.0,26699.56,0.002261,-0.001662,-0.000558,0.000039,0.000018,0.002273,-0.001670,-0.000370,0.003888,0.000146,0.001026,0.023096,0.059366,-0.002955,0.027941,-0.107043,0.015210,0.067555,-0.060716,0.047548,0.000545,0.020629,0.082442,-0.051212,0.014094,-0.058781,-0.119893,0.045242,-0.111781,0.074045,-0.023492,0.073287,0.009238,0.042351,-0.074010,0.057105,0.172568,-0.029542,0.287066,-0.043026,0.063103
NWD283680,77.0,0.0,156.8,70.3080,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5929.0,24586.24,0.002299,-0.001604,-0.000735,0.001416,0.002190,-0.001993,0.002143,0.000234,-0.008070,-0.002662,0.000313,-0.003106,-0.020329,-0.052338,-0.072248,0.008152,0.002561,0.062351,-0.030048,0.044880,0.077665,-0.006763,-0.017742,-0.023913,-0.041030,-0.098488,0.061467,0.004998,-0.074857,-0.123331,0.002699,-0.221857,-0.047416,-0.043757,-0.281043,-0.038263,-0.040050,-0.022498,0.094677,-0.039144,-0.007017
NWD420821,69.0,0.0,154.5,69.6276,0.00,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4761.0,23870.25,0.001329,0.004460,0.006931,-0.000199,0.000409,-0.002876,0.003032,-0.001861,0.003163,-0.003338,0.001002,-0.005017,0.012678,-0.051169,0.015235,0.039495,-0.067349,0.010283,0.021721,-0.000014,0.027525,-0.042953,0.003384,-0.042915,-0.007660,0.079915,0.039471,-0.045643,-0.059135,0.010713,0.007152,-0.077201,0.023902,-0.020847,-0.017434,-0.058831,-0.055029,0.040014,-0.046636,0.219668,-0.006687
NWD638901,81.0,0.0,159.2,58.2876,60.95,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6561.0,25344.64,0.002311,-0.001527,-0.000916,0.001611,0.002563,-0.002922,0.003061,0.000183,-0.012989,-0.005611,0.002007,0.012241,0.005696,-0.068030,-0.068547,-0.062583,-0.022774,0.032830,-0.073796,-0.071511,-0.056976,0.004798,-0.000665,0.061951,-0.034485,-0.055497,-0.028175,-0.049609,-0.092132,-0.132964,0.033360,0.086668,-0.090870,0.068044,-0.090302,0.133982,0.035092,-0.026597,0.039588,0.018415,0.026656


### Features

In [4]:
feature_matrices = []

# load matrices and fill blanks with nan
for matrix in sorted(glob.glob("/home/jve4pt/Manichaikul-PhD-Rotation/rna-seq-regression/output/features///*")): 
   
    tmp_df = pd.read_csv(
        matrix,
        sep="\t"
    ).set_index("Unnamed: 0").replace(["", " "], np.nan)
    
    # get cell type and set as name 
    tmp_df.name = matrix.split("/")[-1].split(".")[0].split("-")[0]
    tmp_df.name
    tmp_df.shape
    tmp_df.head()
    
    feature_matrices.append(tmp_df)

'Monocyte_Exam_5'

(346, 16549)

,WASH7P,ENSG00000268903,ENSG00000269981,ENSG00000241860,WASH9P,RPL23AP21,MTND1P23,MTND2P28,MTCO1P12,MTCO2P12,MTATP8P1,MTATP6P1,MTCO3P12,ENSG00000228327,LINC01409,ENSG00000230092,LINC01128,LINC00115,NOC2L,KLHL17,PLEKHN1,HES4,ISG15,ENSG00000231702,AGRN,C1orf159,TNFRSF4,SDF4,B3GALT6,UBE2J2,SCNN1D,ACAP3,PUSL1,ENSG00000240731,INTS11,CPTP,TAS1R3,DVL1,AURKAIP1,CCNL2,MRPL20-AS1,MRPL20,MRPL20-DT,ANKRD65,ATAD3C,ATAD3B,ATAD3A,SSU72,ENSG00000215014,FNDC10,MIB2,ENSG00000272106,CDK11B,SLC35E2B,CDK11A,SLC35E2A,NADK,GNB1,PRKCZ,ENSG00000271806,PRKCZ-AS1,FAAP20,SKI,LOC100129534,MORN1,RER1,PEX10,PANK4,ENSG00000272449,TNFRSF14,TNFRSF14-AS1,LOC100996583,PRXL2B,MMEL1,TTC34,MEGF6,TPRG1L,WRAP73,TP73-AS1,SMIM1,LRRC47,CEP104,DFFB,C1orf174,LINC01134,KCNAB2,NPHP4,RNF207,RPL22,ICMT,GPR153,ACOT7,TNFRSF25,NOL9,ZBTB48,PHF13,KLHL21,THAP3,DNAJC11,CAMTA1,VAMP3,ENSG00000269925,PER3,UTS2,TNFRSF9,PARK7,ERRFI1,SLC45A1,RERE-AS1,RERE,ENO1-AS1,ENO1,HMGN2P17,SLC2A5,GPR157,H6PD,SPSB1,SLC25A33,RPL9P11,TMEM201,PIK3CD,PIK3CD-AS1,CLSTN1,CTNNBIP1,LZIC,NMNAT1,RBP7,UBE4B,KIF1B,PGD,CENPS,DFFA,PEX14,CASZ1,C1orf127,TARDBP,ENSG00000271895,SRM,EXOSC10-AS1,EXOSC10,RPL39P6,MTOR,UBIAD1,FBXO44,FBXO2,FBXO6,MAD2L2,DRAXIN,AGTRAP,CLCN6,MTHFR,ENSG00000242349,KIAA2013,PLOD1,MFN2,MIIP,TNFRSF8,TNFRSF1B,VPS13D,DHRS3,HNRNPCL1,HNRNPCL2,HNRNPCL4,PRDM2,TMEM51,FHAD1,EFHD2-AS1,EFHD2,CASP9,DNAJC16,AGMAT,CHCHD2P6,DDI2,PLEKHM2,SLC25A34,RPL12P14,UQCRHL,RPS16P1,SPEN,SPEN-AS1,ZBTB17,ARHGEF19,ANO7L1,CPLANE2,FBXO42,SZRD1,RPL22P3,NECAP2,LINC01772,CROCCP3,NBPF1,MST1P2,CROCCP2,EIF1AXP1,CROCC,LOC105376805,ATP13A2,SDHB,PADI2,PADI4,PADI6,RCC2,ARHGEF10L,ALDH4A1,IFFO2,ENSG00000272084,UBR4,EMC1-AS1,MRTO4,EMC1,AKR7L,AKR7A3,AKR7A2,SLC66A1,CAPZB,MICOS10,ENSG00000226396,NBL1,TMCO4,OTUD3,PLA2G2D,LOC117779438,PLA2G2C,UBXN10,ENSG00000225986,MUL1,CDA,PINK1,PINK1-AS,DDOST,KIF17,HP1BP3,EIF4G3,ECE1,NBPF2P,HS6ST1P1,NBPF3,USP48,LDLRAD2,HSPG2,LINC00339,LINC01635,CDC42,ZBTB40,C1QA,C1QC,C1QB,EPHB2,KDM1A,LUZP1,...,EEF1A1P29,RPSAP15,CAPZA1P1,EIF4A1P10,RPL26P36,AP2B1P1,ST13P18,NAP1L3,KAT7P1,DIAPH2,RPSAP8,CSTF2,HNRNPA1P26,HNRNPA1P27,TRMT2B,CENPI,TIMM8A,BTK,RPL36A,GLA,HNRNPH2,ARMCX4,ARMCX1,ARMCX6,ARMCX3,ENSG00000261101,ARMCX2,GK4P,ZMAT1,BEX5,ARMCX5,ARMCX5-GPRASP2,GPRASP1,GPRASP2,BHLHB9,LINC00630,BEX1,NXF3,BEX4,TCEAL8,BEX2,TCEAL9,BEX3,TCEAL4,TCEAL3,TCEAL1,MORF4L2-AS1,MORF4L2,RAB9B,SLC25A53,FAM199X,RADX,TBC1D8B,MORC4,RBM41,GTF3C6P2,DNAJA1P3,PRPS1,TSC22D3,MID2,VSIG1,PSMD10,ATG4A,NXT2,KCNE5,ACSL4,TMEM164,AMMECR1,EIF4BP7,ALG13,AMOT,HSPA8P7,PLS3,EEF1GP5,SUMO2P21,API5P1,KLHL13,WDR44,DOCK11,IL13RA1,TMEM30BP1,LONRF3,HNRNPA1P28,PGRMC1,SLC25A43,SLC25A5,SLC25A5-AS1,STEEP1,UBE2A,NKRF,SEPTIN6,SOWAHD,RPL39,UPF3B,NDUFA1,RNF113A,NKAP,NKAPP1,ZBTB33,TMEM255A,LAMP2,CUL4B,MCTS1,C1GALT1C1,GLUD2,HSPA8P1,RPL3P12,H3P47,THOC2,XIAP,STAG2,SH2D1A,TENM1,MTCO1P53,MTND4P24,OCRL,XPNPEP2,SASH3,ENSG00000240143,ZDHHC9,UTP14A,BCORL1,ELF4,AIFM1,RAB33A,ZNF280C,SLC25A14,RBMX2,DENND10P1,ENOX2,FIRRE,STK26,RAP2C-AS1,RAP2C,MBNL3,GPC4,RPS24P19,PHF6,HPRT1,MIR503HG,PABIR3,PABIR2,MOSPD1,SMIM10,RTL8B,RTL8C,RTL8A,ZNF75D,ZNF449,INTS6L,SLC9A6,MMGT1,FHL1,MAP7D3,HTATSF1,CD40LG,ARHGEF6,ENSG00000232611,RBMX,TM9SF5P,RAC1P4,ATP11C,HNRNPA3P3,LOC389895,RPL36AP52,LDOC1,SLITRK4,UBE2NL,FMR1,FTH1P8,AFF2,IDS,EOLA1,EOLA1-DT,TMEM185A,EOLA2-DT,EOLA2,XRCC6P2,MTM1,MTMR1,CD99L2,HMGB3,RPL19P21,VMA21,CETN2,NSDHL,ZNF185,PNMA5,PNMA3,PNMA6A,ZNF275,ZFP92,TREX2,HAUS7,LOC105373383,CCNQ,RPL18AP16,SLC6A8,BCAP31,ABCD1,SSR4,IDH3G,PDZD4,L1CAM,AVPR2,ARHGAP4,NAA10,RENBP,HCFC1,TMEM187,IRAK1,MECP2,TKTL1,FLNA,EMD,RPL10,ENSG00000280195,SNORA70,TAFAZZIN,DNASE1L1,ATP6AP1-DT,ATP6AP1,GDI1,FAM50A,PLXNA3,LAGE3,UBL4A,SLC10A3,FAM3A,IKBKG,G6PD,IKBKGP1,GAB3,DKC1,MPP1,ENSG00000277203,FUNDC2,F8,BRCC3,MTCP1,VBP1,RAB39B,CLIC2,ENSG00000274791,F8A3,TMLHE-AS1,TMLHE,SPRY3,VAMP7,WASH6P
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

'PBMC_Exam_5'

(850, 19699)

,DDX11L1,WASH7P,ENSG00000238009,ENSG00000268903,ENSG00000269981,ENSG00000241860,DDX11L17,WASH9P,RPL23AP21,RPL23AP24,ENSG00000237094,MTND1P23,MTND2P28,MTCO1P12,MTCO2P12,MTATP8P1,MTATP6P1,MTCO3P12,ENSG00000228327,LINC01409,ENSG00000230092,LINC01128,LINC00115,NOC2L,KLHL17,PLEKHN1,ENSG00000272512,HES4,ISG15,ENSG00000231702,AGRN,LOC100288175,C1orf159,TTLL10,TNFRSF18,TNFRSF4,SDF4,B3GALT6,C1QTNF12,ENSG00000260179,UBE2J2,LINC01786,SCNN1D,ACAP3,PUSL1,ENSG00000240731,INTS11,CPTP,TAS1R3,DVL1,MXRA8,AURKAIP1,CCNL2,MRPL20-AS1,MRPL20,MRPL20-DT,ANKRD65,ATAD3C,ATAD3B,ATAD3A,TMEM240,SSU72,ENSG00000215014,FNDC10,MIB2,ENSG00000272106,MMP23B,CDK11B,ENSG00000272004,SLC35E2B,CDK11A,SLC35E2A,NADK,GNB1,GNB1-DT,CALML6,PRKCZ,ENSG00000271806,PRKCZ-AS1,FAAP20,SKI,LOC100129534,ENSG00000272420,MORN1,RER1,PEX10,PLCH2,PANK4,ENSG00000272449,TNFRSF14,TNFRSF14-AS1,ENSG00000225931,LOC100996583,PRXL2B,MMEL1,TTC34,MEGF6,ENSG00000238260,TPRG1L,TP73,WRAP73,TP73-AS1,SMIM1,LRRC47,CEP104,DFFB,C1orf174,LINC01134,AJAP1,ENSG00000236948,KCNAB2,NPHP4,CHD5,RNF207,RPL22,ICMT,GPR153,ENSG00000271746,ACOT7,ESPN,TNFRSF25,PLEKHG5,NOL9,ZBTB48,PHF13,KLHL21,THAP3,DNAJC11,CAMTA1,VAMP3,ENSG00000269925,PER3,ENSG00000236266,UTS2,TNFRSF9,PARK7,ERRFI1,SLC45A1,RERE-AS1,RERE,ENSG00000236269,ENO1-AS1,ENO1,HMGN2P17,ENSG00000232208,CA6,SLC2A5,GPR157,MIR34AHG,H6PD,SPSB1,SLC25A33,RPL9P11,TMEM201,PIK3CD,PIK3CD-AS1,PIK3CD-AS2,CLSTN1,CTNNBIP1,LZIC,NMNAT1,RBP7,UBE4B,KIF1B,PGD,CENPS,DFFA,PEX14,CASZ1,C1orf127,TARDBP,ENSG00000271895,SRM,ENSG00000226849,EXOSC10,RPL39P6,MTOR,UBIAD1,DISP3,FBXO44,FBXO2,FBXO6,MAD2L2,DRAXIN,AGTRAP,CLCN6,MTHFR,ENSG00000242349,KIAA2013,PLOD1,MFN2,MIIP,TNFRSF8,TNFRSF1B,VPS13D,LINC02766,DHRS3,HNRNPCL1,HNRNPCL2,PRDM2,TMEM51,FHAD1,FHAD1-AS1,EFHD2-AS1,EFHD2,CTRC,CASP9,DNAJC16,ENSG00000272510,AGMAT,CHCHD2P6,DDI2,PLEKHM2,SLC25A34,RPL12P14,UQCRHL,RPS16P1,SPEN,SPEN-AS1,ZBTB17,EPHA2,ARHGEF19,ANO7L1,CPLANE2,FBXO42,SZRD1,RPL22P3,NECAP2,LINC01772,CROCCP3,ENSG00000261135,NBPF1,MST1P2,CROCCP2,EIF1AXP1,ESPNP,CROCC,MST1L,LOC105376805,ATP13A2,SDHB,PADI2,PADI4,PADI6,RCC2,ARHGEF10L,ALDH4A1,IFFO2,ENSG00000272084,UBR4,...,TCEAL2,BEX5,TMSB15A,ARMCX5,ARMCX5-GPRASP2,GPRASP1,GPRASP2,BHLHB9,LINC00630,ENSG00000239407,BEX1,NXF3,BEX4,TCEAL8,BEX2,TCEAL9,BEX3,TCEAL4,TCEAL3,TCEAL1,MORF4L2-AS1,MORF4L2,RAB9B,ELF2P1,TMSB15B,TMSB15B-AS1,H2BW3P,LOC122394733,ZCCHC18,SLC25A53,FAM199X,RADX,TBC1D8B,MORC4,RBM41,GTF3C6P2,NUP62CL,DNAJA1P3,FRMPD3,PRPS1,NCBP2L,TSC22D3,MID2,VSIG1,PSMD10,ATG4A,NXT2,KCNE5,ACSL4,TMEM164,AMMECR1,PAK3,EIF4BP7,ALG13,LHFPL1,AMOT,HSPA8P7,PLS3,EEF1GP5,SUMO2P21,API5P1,KLHL13,WDR44,DOCK11,IL13RA1,TMEM30BP1,LONRF3,HNRNPA1P28,PGRMC1,SLC25A43,SLC25A5,SLC25A5-AS1,STEEP1,UBE2A,NKRF,SEPTIN6,SOWAHD,RPL39,UPF3B,NDUFA1,RNF113A,NKAP,RHOXF1-AS1,RHOXF1,EEF1A1P30,NKAPP1,ZBTB33,TMEM255A,LAMP2,CUL4B,MCTS1,C1GALT1C1,GLUD2,HSPA8P1,RPL3P12,H3P47,THOC2,XIAP,STAG2,STAG2-AS1,SH2D1A,TENM1,MTCO1P53,MTND4P24,SMARCA1,OCRL,XPNPEP2,SASH3,ENSG00000240143,ZDHHC9,UTP14A,BCORL1,ELF4,AIFM1,RAB33A,ZNF280C,SLC25A14,RBMX2,DENND10P1,ENOX2,FIRRE,STK26,RAP2C-AS1,RAP2C,MBNL3,GPC4,RPS24P19,GPC3,PHF6,HPRT1,MIR503HG,PABIR3,PABIR2,MOSPD1,SMIM10,RTL8B,RTL8C,RTL8A,ZNF75D,ZNF449,SMIM10L2A,INTS6L,SLC9A6,MMGT1,FHL1,MAP7D3,HTATSF1,LINC00892,CD40LG,ARHGEF6,RANP4,ENSG00000232611,RBMX,TM9SF5P,MCF2,ATP11C,HNRNPA3P3,LOC389895,RPL36AP52,LDOC1,SLITRK4,UBE2NL,ANKRD11P2,FMR1,FMR1-IT1,FTH1P8,AFF2,IDS,ENSG00000238039,EOLA1,EOLA1-DT,TMEM185A,EOLA2-DT,EOLA2,LOC643015,XRCC6P2,MAMLD1,MTM1,MTMR1,CD99L2,HMGB3,RPL19P21,VMA21,CETN2,NSDHL,ZNF185,PNMA5,PNMA3,PNMA6A,ZNF275,ZFP92,TREX2,HAUS7,LOC105373383,CCNQ,RPL18AP16,SLC6A8,BCAP31,ABCD1,PLXNB3,SSR4,IDH3G,PDZD4,L1CAM,AVPR2,ARHGAP4,NAA10,RENBP,HCFC1,TMEM187,IRAK1,MECP2,TKTL1,FLNA,EMD,RPL10,ENSG00000280195,SNORA70,TAFAZZIN,DNASE1L1,ATP6AP1-DT,ATP6AP1,GDI1,FAM50A,PLXNA3,LAGE3,UBL4A,SLC10A3,FAM3A,IKBKG,G6PD,IKBKGP1,GAB3,DKC1,MPP1,ENSG00000277203,FUNDC2,F8,BRCC3,MTCP1,VBP1,RAB39B,CLIC2,ENSG00000274791,F8A3,TMLHE,SPRY3,VAMP7,IL9R,WASH6P
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

'T_cell_Exam_5'

(353, 19167)

,WASH7P,ENSG00000268903,ENSG00000269981,ENSG00000241860,WASH9P,RPL23AP21,RPL23AP24,ENSG00000237094,MTND1P23,MTND2P28,MTCO1P12,MTCO2P12,MTATP8P1,MTATP6P1,MTCO3P12,ENSG00000228327,LINC01409,ENSG00000230092,LINC01128,LINC00115,NOC2L,KLHL17,PLEKHN1,HES4,ISG15,ENSG00000231702,AGRN,LOC100288175,C1orf159,TTLL10,TNFRSF18,TNFRSF4,SDF4,B3GALT6,C1QTNF12,ENSG00000260179,UBE2J2,LINC01786,SCNN1D,ACAP3,PUSL1,ENSG00000240731,INTS11,CPTP,TAS1R3,DVL1,MXRA8,AURKAIP1,CCNL2,MRPL20-AS1,MRPL20,MRPL20-DT,ANKRD65,ATAD3C,ATAD3B,ATAD3A,TMEM240,SSU72,ENSG00000215014,FNDC10,MIB2,ENSG00000272106,CDK11B,SLC35E2B,CDK11A,SLC35E2A,NADK,GNB1,GNB1-DT,CALML6,PRKCZ,ENSG00000271806,PRKCZ-AS1,FAAP20,SKI,LOC100129534,ENSG00000272420,MORN1,RER1,PEX10,PLCH2,PANK4,ENSG00000272449,TNFRSF14,TNFRSF14-AS1,ENSG00000225931,LOC100996583,PRXL2B,MMEL1,TTC34,ARHGEF16,MEGF6,ENSG00000238260,TPRG1L,TP73,WRAP73,TP73-AS1,SMIM1,ENSG00000272153,LRRC47,CEP104,DFFB,C1orf174,LINC01134,AJAP1,ENSG00000236948,KCNAB2,NPHP4,CHD5,RNF207,RPL22,ICMT,GPR153,ENSG00000271746,ACOT7,ESPN,TNFRSF25,PLEKHG5,NOL9,ZBTB48,PHF13,KLHL21,THAP3,DNAJC11,CAMTA1,VAMP3,ENSG00000269925,PER3,ENSG00000236266,UTS2,TNFRSF9,PARK7,ERRFI1,SLC45A1,RERE-AS1,RERE,ENO1-AS1,ENO1,HMGN2P17,ENSG00000232208,CA6,SLC2A5,GPR157,LNCTAM34A,MIR34AHG,H6PD,SPSB1,SLC25A33,RPL9P11,TMEM201,PIK3CD,PIK3CD-AS1,PIK3CD-AS2,CLSTN1,CTNNBIP1,LZIC,NMNAT1,RBP7,UBE4B,KIF1B,PGD,CENPS,DFFA,PEX14,CASZ1,C1orf127,TARDBP,ENSG00000271895,SRM,ENSG00000226849,EXOSC10,RPL39P6,MTOR,UBIAD1,DISP3,FBXO44,FBXO2,FBXO6,MAD2L2,DRAXIN,AGTRAP,CLCN6,MTHFR,ENSG00000242349,KIAA2013,PLOD1,MFN2,MIIP,TNFRSF8,TNFRSF1B,VPS13D,LINC02766,DHRS3,HNRNPCL1,HNRNPCL2,HNRNPCL4,PRDM2,TMEM51,EFHD2-AS1,EFHD2,CTRC,CASP9,DNAJC16,ENSG00000272510,AGMAT,CHCHD2P6,DDI2,PLEKHM2,SLC25A34,RPL12P14,UQCRHL,RPS16P1,SPEN,SPEN-AS1,ZBTB17,EPHA2,ARHGEF19,ANO7L1,CPLANE2,FBXO42,SZRD1,RPL22P3,NECAP2,LINC01772,CROCCP3,ENSG00000261135,NBPF1,MST1P2,CROCCP2,EIF1AXP1,ESPNP,CROCC,MST1L,LOC105376805,ATP13A2,SDHB,PADI2,PADI4,PADI6,RCC2,ARHGEF10L,ALDH4A1,IFFO2,ENSG00000272084,UBR4,EMC1-AS1,MRTO4,EMC1,AKR7L,AKR7A3,...,ARMCX5-GPRASP2,GPRASP1,GPRASP2,BHLHB9,LINC00630,ENSG00000239407,BEX1,BEX4,TCEAL8,BEX2,TCEAL9,BEX3,TCEAL4,TCEAL3,TCEAL1,MORF4L2-AS1,MORF4L2,RAB9B,ELF2P1,TMSB15B,TMSB15B-AS1,SLC25A53P1,H2BW3P,LOC122394733,ZCCHC18,SLC25A53,FAM199X,RADX,RNF128,TBC1D8B,MORC4,RBM41,GTF3C6P2,NUP62CL,DNAJA1P3,FRMPD3,PRPS1,TSC22D3,MID2,VSIG1,PSMD10,ATG4A,NXT2,KCNE5,ACSL4,TMEM164,AMMECR1,PAK3,EIF4BP7,ALG13,LHFPL1,AMOT,HSPA8P7,PLS3,EEF1GP5,SUMO2P21,API5P1,KLHL13,WDR44,DOCK11,IL13RA1,TMEM30BP1,LONRF3,HNRNPA1P28,PGRMC1,SLC25A43,SLC25A5,SLC25A5-AS1,STEEP1,UBE2A,NKRF,SEPTIN6,SOWAHD,RPL39,UPF3B,NDUFA1,RNF113A,NKAP,RHOXF1,EEF1A1P30,NKAPP1,ZBTB33,TMEM255A,LAMP2,CUL4B,MCTS1,C1GALT1C1,PA2G4P1,GLUD2,HSPA8P1,RPL3P12,GRIA3,H3P47,THOC2,XIAP,STAG2,SH2D1A,TENM1,MTCO1P53,MTND4P24,SMARCA1,OCRL,XPNPEP2,SASH3,ENSG00000240143,ZDHHC9,UTP14A,BCORL1,ELF4,AIFM1,RAB33A,ZNF280C,SLC25A14,RBMX2,DENND10P1,ENOX2,FIRRE,STK26,RAP2C-AS1,RAP2C,MBNL3,GPC4,RPS24P19,GPC3,PHF6,HPRT1,PABIR3,PABIR2,MOSPD1,SMIM10,RTL8B,RTL8C,RTL8A,SMIM10L2B,ZNF75D,ZNF449,SMIM10L2A,INTS6L,SLC9A6,MMGT1,FHL1,MAP7D3,HTATSF1,LINC00892,CD40LG,ARHGEF6,RPL7P56,RANP4,ENSG00000232611,RBMX,TM9SF5P,SRRM1P3,MCF2,ATP11C,HNRNPA3P3,LOC389895,RPL36AP52,LDOC1,SLITRK4,UBE2NL,ANKRD11P2,FMR1,FMR1-IT1,FTH1P8,AFF2,IDS,ENSG00000238039,EOLA1,EOLA1-DT,TMEM185A,HSFX1,EOLA2-DT,EOLA2,LOC643015,XRCC6P2,MAMLD1,MTM1,MTMR1,CD99L2,HMGB3,RPL19P21,VMA21,CETN2,NSDHL,ZNF185,PNMA5,PNMA3,PNMA6A,ZNF275,ZFP92,TREX2,HAUS7,LOC105373383,CCNQ,RPL18AP16,SLC6A8,BCAP31,ABCD1,SSR4,IDH3G,PDZD4,L1CAM,AVPR2,ARHGAP4,NAA10,RENBP,HCFC1,TMEM187,IRAK1,MECP2,TKTL1,FLNA,EMD,RPL10,ENSG00000280195,SNORA70,TAFAZZIN,DNASE1L1,ATP6AP1-DT,ATP6AP1,GDI1,FAM50A,PLXNA3,LAGE3,UBL4A,SLC10A3,FAM3A,ENSG00000261773,IKBKG,G6PD,IKBKGP1,GAB3,DKC1,HMGN1P37,MPP1,ENSG00000277203,FUNDC2,F8,BRCC3,MTCP1,VBP1,RAB39B,CLIC2,ENSG00000274791,F8A3,TMLHE,SPRY3,VAMP7,IL9R,WASH6P
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

## Expression Testing

### Hit Genes

Testing for genes that were affected by colocalized variants that were signicant between (e/m/p)QTL datasets. 

In [5]:
hit_genes = ["HLA-DQA2", "GSTCD", "ADAM19"]
targets=  ["pre_fev1fvc", "pre_fev1", "pre_fev1fvc"]


In [6]:
# intialize dataframe 
hit_testing_df = pd.DataFrame(columns=["Trait", "# Participants", "Cell Type", "Gene", "Raw P-val" ])


# for each cell type 
for counter_1 in range(len(target_matrices)):
    # for each of the genes and targets 
    for counter_2 in range(len(hit_genes)): 
        
        # subset cell type specific target matrix by target variable 
        # drop missing values to get target samples
        # inner join this with cell type specific covariate matrix
        # inner join with cell type specific feature matrix for specific gene 
        regression_df = target_matrices[counter_1][
            [targets[counter_2]]
        ].dropna().join(
            covariate_matrices[counter_1], 
            how="inner"
        ).join(
            feature_matrices[counter_1][[hit_genes[counter_2]]], 
            how="inner"
        )
        
        # convert all columns to float type by creating dict from tuple of column name and "float" 
        regression_df = regression_df.astype(
            dict(
                zip(
                    regression_df.columns.to_list(),
                    [ "float" for x in range(regression_df.columns.size)]
                )
            )
        )
        
        # create target and feature dfs
        # run Ordinary Least Squares Regression 
        results = statsmodels.regression.linear_model.OLS(
            regression_df[targets[counter_2]], 
            regression_df.iloc[:,1:]
        ).fit()
        
        # create list for new row to append info about each test to `hit_testing_df`
        hit_testing_df.loc[hit_testing_df.index.size] = [
            targets[counter_2], 
            regression_df.index.size, 
            target_matrices[counter_1].name, 
            results.pvalues.index.to_list()[-1],
            results.pvalues[-1]
        ]
        
        

In [7]:
hit_testing_df.sort_values("Raw P-val")

,Trait,# Participants,Cell Type,Gene,Raw P-val
7,pre_fev1,271,T_cell_Exam_5,GSTCD,0.041905
3,pre_fev1fvc,631,PBMC_Exam_5,HLA-DQA2,0.157132
6,pre_fev1fvc,270,T_cell_Exam_5,HLA-DQA2,0.237175
0,pre_fev1fvc,259,Monocyte_Exam_5,HLA-DQA2,0.356288
8,pre_fev1fvc,270,T_cell_Exam_5,ADAM19,0.399053
1,pre_fev1,259,Monocyte_Exam_5,GSTCD,0.498476
2,pre_fev1fvc,259,Monocyte_Exam_5,ADAM19,0.641643
4,pre_fev1,632,PBMC_Exam_5,GSTCD,0.657157
5,pre_fev1fvc,631,PBMC_Exam_5,ADAM19,0.832049


### All Genes

In [ ]:
# list for saving dataframes of p-values per gene for each combination of cell type and target variable
testing_dfs = []

# for each cell type 
for counter in range(len(target_matrices)):
    # for each target variable 
    for target in target_matrices[counter].columns.to_list(): 
        # intialize dataframe 
        all_testing_df = pd.DataFrame(columns=["Trait", "# Participants", "Cell Type", "Gene", "Raw P-val" ])

        # for each gene 
        for gene in feature_matrices[counter].columns.to_list(): 
        
            # subset cell type specific target matrix by target variable 
            # drop missing values to get target samples
            # inner join this with cell type specific covariate matrix
            # inner join with cell type specific feature matrix for specific gene 
            regression_df = target_matrices[counter][
                [target]
            ].dropna().join(
                covariate_matrices[counter], 
                how="inner"
            ).join(
                feature_matrices[counter][[gene]], 
                how="inner"
            )

            # convert all columns to float type by creating dict from tuple of column name and "float" 
            regression_df = regression_df.astype(
                dict(
                    zip(
                        regression_df.columns.to_list(),
                        [ "float" for x in range(regression_df.columns.size)]
                    )
                )
            )

            # create target and feature dfs
            # run Ordinary Least Squares Regression 
            results = statsmodels.regression.linear_model.OLS(
                regression_df[[target]], 
                regression_df.iloc[:,1:]
            ).fit()

            # create list for new row to append info about each test to `hit_testing_df`
            all_testing_df.loc[all_testing_df.index.size] = [
                target, 
                regression_df.index.size, 
                target_matrices[counter].name, 
                results.pvalues.index.to_list()[-1],
                results.pvalues[-1]
            ]
            
        # set name to cell type and target variable
        all_testing_df.name = "{}-{}".format(
            target_matrices[counter].name, 
            target
        )
        
        testing_dfs.append(all_testing_df)
        
        

## Multiple Hypothesis Correction

### FDR Correction

In [45]:
for testing_df in testing_dfs: 
    testing_df["FDR P-val"] = statsmodels.stats.multitest.multipletests(
        testing_df["Raw P-val"], 
        method = "fdr_bh"
    )[1]

### Bonferroni Correction

In [46]:
for testing_df in testing_dfs: 
    testing_df["Bonferroni P-val"] = statsmodels.stats.multitest.multipletests(
        testing_df["Raw P-val"], 
        method = "bonferroni"
    )[1]

In [57]:
tmp = []
for thing in target_matrices: 
    tmp.extend([thing.name] * 6)

In [58]:
tmp

['Monocyte_Exam_5',
 'Monocyte_Exam_5',
 'Monocyte_Exam_5',
 'Monocyte_Exam_5',
 'Monocyte_Exam_5',
 'Monocyte_Exam_5',
 'PBMC_Exam_5',
 'PBMC_Exam_5',
 'PBMC_Exam_5',
 'PBMC_Exam_5',
 'PBMC_Exam_5',
 'PBMC_Exam_5',
 'T_cell_Exam_5',
 'T_cell_Exam_5',
 'T_cell_Exam_5',
 'T_cell_Exam_5',
 'T_cell_Exam_5',
 'T_cell_Exam_5']

## Combine All Tables

In [61]:
final_testing_df = pd.concat(testing_dfs)

final_testing_df.shape
final_testing_df.head()

(332490, 7)

,Trait,# Participants,Cell Type,Gene,Raw P-val,FDR P-val,Bonferroni P-val
0,pre_fev1,259,Monocyte_Exam_5,WASH7P,0.388175,0.972646,1.0
1,pre_fev1,259,Monocyte_Exam_5,ENSG00000268903,0.581526,0.987966,1.0
2,pre_fev1,259,Monocyte_Exam_5,ENSG00000269981,0.578854,0.987966,1.0
3,pre_fev1,259,Monocyte_Exam_5,ENSG00000241860,0.254166,0.955073,1.0
4,pre_fev1,259,Monocyte_Exam_5,WASH9P,0.610872,0.987966,1.0


In [62]:
final_testing_df.sort_values("Bonferroni P-val").head(n=20)

,Trait,# Participants,Cell Type,Gene,Raw P-val,FDR P-val,Bonferroni P-val
794,pre_fvc,270,T_cell_Exam_5,IFI44,0.000002,0.028980,0.037005
1953,pre_fvc,270,T_cell_Exam_5,RSAD2,0.000003,0.028980,0.057960
4580,pre_fev1,271,T_cell_Exam_5,HSD17B11,0.000003,0.065205,0.065205
7817,post_fev1fvc,111,T_cell_Exam_5,NUDT18,0.000005,0.103568,0.103568
7634,post_fev1fvc,107,Monocyte_Exam_5,ATP6V1G1,0.000007,0.122416,0.122416
5116,post_fev1fvc,243,PBMC_Exam_5,TRIP13,0.000008,0.129466,0.160170
8537,pre_fev1fvc,631,PBMC_Exam_5,ZFAT,0.000009,0.167743,0.167743
15259,post_fev1,243,PBMC_Exam_5,RPL21P120,0.000010,0.096810,0.188181
8951,post_fev1,243,PBMC_Exam_5,SPTLC1,0.000010,0.096810,0.193619
18172,post_fvc,243,PBMC_Exam_5,KCNG1,0.000011,0.225487,0.225487


In [63]:
final_testing_df.sort_values("Bonferroni P-val").to_csv(
    "/home/jve4pt/Manichaikul-PhD-Rotation/rna-seq-regression/output/results/all_genes_analysis.tsv", 
    sep="\t",
    index=False
)